In [1]:
%%time
import pal
import json
import random
import math
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt
from tqdm.notebook import tqdm

CPU times: user 501 ms, sys: 180 ms, total: 681 ms
Wall time: 857 ms


In [2]:
#IMPORTANT: YOU DO NOT WRITE OBSERVATIONS. THEY WILL BE PROVIDED TO YOU. 
prompt = """Whenever you synthesize information and perform reasoning, it is a Thought. When you need to calculate a value, e.g., 5+3, it is an Action. There are 6 types of actions: 
(1)ADD[a;b]: ADD takes in two numbers a and b, and returns their sum. For example, ADD[7;3] will return the value of 7+3.
(2)SUB[a;b]: SUB takes in two numbers a and b, and returns their difference. For example, SUB[6;2] will return the value of 6-2.
(3)MULT[a;b]: MULT takes in two numbers a and b, and returns their product. For example, MULT[8;5] will return the value of 8*5.
(4)DIV[a;b]: DIV takes in two numbers a and b, and returns their quotient. For example, DIV[9;3] will return 9/3.
(5)SQRT[a]: SQRT takes in a number a, and returns its square root. For example, SQRT[25] will return the square root of 25, i.e., 5.
(6)FINISH[answer]FINISH: When you finish a question, write your answer as FINISH[answer] and terminate your response.
Terminate your response after an action. Then, an Observation is returned by the user. You will be given a question and you will perform Thoughts and Actions until you get the answer. When you get the answer, you will perform the Action called Finish and write Finish[answer]. While you are reasoning out solutions, you will continually refer back to relevant portions of the question and employ logical reasoning to determine your next action."""
mario_luigi = [{"role": "user", "content": prompt + '\n' + "Question: Mario and Luigi together had 11 years of experience in soccer. Luigi had 3 more than Mario. How many did Mario have?"},
        {"role": "assistant", "content": """Thought: Let M be the number of years of experience Mario had. Luigi had M + 3 years of experience. The question states that together they had 11 years of experience, so M + (M + 3)=11. We should first simplify this by grouping the M together. So we want (1+1)M + 3=11.
Action: CALCULATE[1 + 1]"""},
        {"role": "user", "content": "Observation: 2"},
        {"role": "assistant", "content": """Thought: The coefficient of M is 2, so we have the equation 2M+3=11. To solve this equation, we first have to subtract 3 from both sides, so 2M = 11 - 3
Action: CALCULATE[11 - 3]"""},
        {"role": "user", "content": "Observation: 8"},
        {"role": "assistant", "content": """Thought: So we have 2M = 8. We can now divide both sides by 2 to get M = 8 / 2
Action: CALCULATE[8 / 2]"""},
        {"role": "user", "content": "Observation: 4"},
        {"role": "assistant", "content": """Thought: So M = 4. Therefore Mario had 4 years of experience
Action: FINISH[4]"""}]

In [3]:
gsm_messages = [
        {"role": "system", "content": "You are a helpful assistant that accurately solves mathematical reasoning questions by acting as an agent who will perform Thoughts and Actions."},
#        {"role": "user", "content": prompt + '\n' + "Question: Lisa is looking to attempt a World Record. She has decided to try and match Joey Chestnut's record of eating 75 full hotdogs, buns included, in 10 minutes. Halfway through the time Lisa has eaten 20 hotdogs. How many hotdogs will she have to eat per minute to at least tie Joey Chestnut's record?"},
#        {"role": "assistant", "content": """Thought: Joey Chestnut ate 75 hotdogs to claim the record and Lisa has eaten 20 hot dogs so far, so she still needs to eat 75 - 20 hotdogs to tie Joey Chestnut.
#Action: SUB[75;20]"""},
#        {"role": "user", "content": "Observation: 55"},
#        {"role": "assistant", "content": """Thought: Lisa still has to eat 55 hotdogs. Lisa has a 10 minute time period to eat the hotdogs and half the time has already passed, which means Lisa has 10/2 minutes left.
#Action: DIV[10;2]"""},
#        {"role": "user", "content": "Observation: 5"},
#        {"role": "assistant", "content": """Thought: So Lisa has 5 minutes left to eat 55 hotdogs. Therefore, she needs to eat 55/5 hotdogs per minute to tie.
#Action: DIV[55;5]"""},
#        {"role": "user", "content": "Observation: 11"},
#        {"role": "assistant", "content": """Thought: Therefore, Lisa has to eat 11 hotdogs per minute to tie Joey Chestnut's record.
#Action: FINISH[11]"""},
        {"role": "user", "content": prompt + '\n' + "Question: Joseph and his friends watched two movies in his house. The first movie is 1 hour and 30 minutes long while the second movie is 30 minutes longer than the first. Before the movies, they spent 10 minutes making popcorn and twice as long making fries. How long, in hours, did it take Joseph and his friends to cook and watch the movies?"},
        {"role": "assistant", "content": """Thought: An hour is 60 minutes, so the first movie was 60 + 30 minutes long.
Action: ADD[60;30]"""},
        {"role": "user", "content": "Observation: 90"},
        {"role": "assistant", "content": """Thought: So the first movie was 90 minutes long. Therefore, the second movie is 90 + 30 minutes long.
Action: ADD[90;30]"""},
        {"role": "user", "content": "Observation: 120"},
        {"role": "assistant", "content": """Thought: So the second movie is 120 minutes long. It took them a total of 90 + 120 minutes to watch the two movies.
Action: ADD[90;120]"""},
        {"role": "user", "content": "Observation: 210"},
        {"role": "assistant", "content": """Thought: Thus, it took them 210 minutes to watch the two movies. It also took twice as long making fries as making popcorn, and since it took 10 minutes to make popcorn, it took 10 * 2 minutes to make fries.
Action: MULT[10;2]"""},
        {"role": "user", "content": "Observation: 20"},
        {"role": "assistant", "content": """Thought: Therefore, it took 20 minutes to make the fries. Since it took 10 minutes to make the popcorn, in total, it took 210 + 20 + 10 minutes to cook and watch the movies. We first find 210 + 20.
Action: ADD[210;20]"""},
        {"role": "user", "content": "Observation: 230"},
        {"role": "assistant", "content": """Thought: Therefore, 210 + 20 + 10 equals 230 + 10. Now we find 230 + 10.
Action: ADD[230;10]"""},
        {"role": "user", "content": "Observation: 240"},
        {"role": "assistant", "content": """Thought: Thus, it took 240 minutes in total to cook and watch the movies. Finally, the problem asks for how many hours it took. Each hour is 60 minutes, so we divide 240 by 60 to get the number of hours.
Action: DIV[240;60]"""},
        {"role": "user", "content": "Observation: 4"},
        {"role": "assistant", "content": """Thought: So it took 4 hours in total to cook and watch the movies.
Action: FINISH[4]"""}
            ]
#for message in gsm_messages:
#    print(message["role"].upper())
#    print(message["content"])
#    print("")

In [4]:
aqua_prompt = prompt + " Remember: Your returned answer MUST be one of the letter answer choices, e.g., (a), (b), (c), (d), or (e)."
aqua_messages = [{"role": "user", "content": aqua_prompt + '\n' + "Question: In the county of Veenapaniville, there are a total of 50 high schools, of three kinds: 25 public schools, 16 parochial schools, and 9 private independent schools. These 50 schools are divided between three districts: A, B, and C. District A has 18 high schools total. District B has 17 high schools total, and only two of those are private independent schools. If District C has an equal number of each of the three kinds of schools, how many private independent schools are there in District A?Answer Choices:(a) 2(b) 3(c) 4(d) 5(e) 6"},
                 {"role": "assistant", "content": """Thought: We should first find the number of schools in District C. Since there are 50 schools in total, and District A has 18 schools in total, and District B has 17 schools in total, District C must have 50-18-17 schools in total.We first calculate 50-18.
Action: SUB[50;18]"""},
                 {"role": "user", "content": "Observation: 32"},
                 {"role": "assistant", "content": """Thought: Therefore, 50-18-17 equals 32-17, so we now calculate 32-17.
Action: SUB[32;17]"""},
                 {"role": "user", "content": "Observation: 15"},
                 {"role": "assistant", "content": """Thought: Therefore, 50-18-17 equals 32-17 equals 15, so District C must have 15 students. District C has an equal number of each of the three kinds of schools, so it has 15/3 of each kind.
Action: DIV[15;3]"""},
                 {"role": "user", "content": "Observation: 5"},
                 {"role": "assistant", "content": """Thought: This means that District C has 5 of each kind. Now we want to find the number of private independent schools in District A. We are given in the question that there are 9 such schools in total. Further, we are given that there are 2 such schools in District B and we just found that there are 5 such schools in District C. Thus, there are 9-2-5 such schools in District A. We first calculate 9-2.
Action: SUB[9;2]"""},
                 {"role": "user", "content": "Observation: 7"},
                 {"role": "assistant", "content": """Thought: Therefore, 9-2-5 equals 7-5, so we now calculate 7-5.
Action: SUB[7;5]"""},
                 {"role": "user", "content": "Observation: 2"},
                 {"role": "assistant", "content": """Thought: Thus, 9-2-5 equals 7-5 equals 2, so there must be 2 private independent schools in District A. 2 corresponds to Answer Choice (a), so our answer is (a).
Action: FINISH[(a)]"""},
                 {"role": "user", "content": aqua_prompt + '\n' + "Question: Two dogsled teams raced across a 300 mile course in Wyoming. Team A finished the course in 3 fewer hours than team T. If team A's average speed was 5 mph greater than team T's, what was team T's average mph?Answer Choices:(a) 12(b) 15(c) 18(d) 20(e) 25"},
                 {"role": "assistant", "content": """Thought: Say Team T's speed was s. Since time equals distance over speed, Team T's time was \\[\\frac{300}{s}\\]. Team A's speed was 5 mph more than Team T's, so their speed was s+5, so Team A's time was \\[\\frac{300}{s+5}\\]. Since Team A finished the course in 3 fewer hours, this means that 
\\[\\frac{300}{s+5}=\\frac{300}{s}-3\\]
Multiplying by \\[s(s+5)\\] on both sides yields
\\[ 300s=300(s+5)-3s(s+5)\\]
which rearranges to
\\[ 300s=300s+300\\cdot 5-3s^2-3\\cdot 5s\\]
First, we evaluate \\[300\\cdot 5\\].
Action: MULT[300;5]"""},
                 {"role": "user", "content": "Observation: 1500"},
                 {"role": "assistant", "content": """Thought: So 300*5=1500. Thus, our equation is
\\[ 300s=300s+1500-3s^2-3\\cdot 5s\\]
Now we evaluate \\[3\\cdot 5\\]
Action: MULT[3;5]"""},
                 {"role": "user", "content": "Observation: 15"},
                 {"role": "assistant", "content": """Thought: Therefore, \\[3\\cdot 5\\] is 15, so our equation is
\\[ 300s=300s+1500-3s^2-15s\\]
Now the 300s on both sides cancel out, so we're left with
\\[0=1500-3s^2-15s\\]
which rearranges to
\\[3s^2+15s-1500=0\\]
We can now use the quadratic formula, which gives that
\\[s=\\frac{-15\\pm\\sqrt{15^2-4(3)(-1500)}}{2\\cdot 3}\\]
We first calculate \\[15^2\\]
Action: MULT[15;15]"""},
                 {"role": "user", "content": "Observation: 225"},
                 {"role": "assistant", "content": """Thought: Therefore, our expression equals
\\[\\frac{-15\\pm\\sqrt{225-4(3)(-1500)}}{2\\cdot 3}\\]
We now calculate \\[4\\cdot 3\\].
Action: MULT[4;3]"""},
                 {"role": "user", "content": "Observation: 12"},
                 {"role": "assistant", "content": """Thought: Thus, our expression equals
\\[\\frac{-15\\pm\\sqrt{225-12(-1500)}}{2\\cdot 3}\\]
Now since we are subtracting 12 times -1500, this is equivalent to adding \\[12\\cdot 1500\\], so our expression is equivalent to
\\[\\frac{-15\\pm\\sqrt{225+12\\cdot 1500}}{2\\cdot 3}\\]
We now find \\[12\\cdot 1500\\]
Action: MULT[12;1500]"""},
                 {"role": "user", "content": "Observation: 18000"},
                 {"role": "assistant", "content": """Thought: Thus, our expression equals
\\[\\frac{-15\\pm\\sqrt{225+18000}}{2\\cdot 3}\\]
We now add 225 with 18000.
Action: ADD[225;18000]"""},
                 {"role": "user", "content": "Observation: 18225"},
                 {"role": "assistant", "content": """Thought: So our expression equals
\\[\\frac{-15\\pm\\sqrt{18225}}{2\\cdot 3}\\]
We need to find the square root of 18225.
Action: SQRT[18225]"""},
                 {"role": "user", "content": "Observation: 135"},
                 {"role": "assistant", "content": """Thought: Thus, our expression equals
\\[\\frac{-15\\pm 135}{2\\cdot 3}\\]
Now since the speed cannot be negative, we take the positive solution, so
\\[s=\\frac{-15+135}{2\\cdot 3}\\]
We now find -15+135.
Action: SUB[135;15]"""},
                 {"role": "user", "content": "Observation: 120"},
                 {"role": "assistant", "content": """Thought: Therefore, our expression equals
\\[\\frac{120}{2\\cdot 3}\\]
We have to find \\[2\\cdot 3\\].
Action: MULT[2;3]"""},
                 {"role": "user", "content": "Observation: 6"},
                 {"role": "assistant", "content": """Thought: Thus, our expression is equal to \\[\\frac{120}{6}\\], so to finish, we divide 120 by 6.
Action: DIV[120;6]"""},
                 {"role": "user", "content": "Observation: 20"},
                 {"role": "assistant", "content": """Thought: Thus, our speed is 20. This corresponds to answer choice (d), so our final answer is (d)
Action: FINISH[(d)]"""}
                ]
                 

In [45]:
interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)

json_list = None
max_count = 100
counter = 0
responses = []
with open('datasets/gsm.jsonl', 'r') as json_file:
    json_list = list(json_file)

jsonSamples = random.sample(range(0, len(json_list)), max_count)
#jsonSamples = [819, 1000, 699, 749, 580, 366, 1275, 203, 492, 773]
#jsonSamples = [269, 436, 836, 1291, 1283, 712, 391, 474, 1164, 783]
#jsonSamples = [1181]
#jsonSamples = [115, 100, 138, 15, 155]
#jsonSamples = [1288]
maxIters = 20
print(jsonSamples)

for json_pos in tqdm(range(len(json_list)), desc="Going through math questions"):
    result = json.loads(json_list[json_pos])
    question = result["input"]
    target = result["target"]
    curResponse = {"question": question, "target": target}
    messages = []
    for message in gsm_messages:
        messages.append(message)
    messages.append({"role": "user", "content": prompt + '\n' + "Question: " + question})
    # interface.verbose = True
    finished = False
    numIters = 0
    #print(question)
    #print("Target: " + str(target))
    while not finished:
        if numIters >= maxIters:
            print("##############################################")
            print("Maximum Number of Iterations Reached")
            curResponse["received"] = -987654321
            break
        numIters += 1
        gens = None
        for i in range(10):
            try:
                gens = interface.generate_history(messages=messages, max_tokens=1024)
                break
            except Exception as e:
                print(counter)
                print(e)
        output = gens[0]
        #print("Output:")
        #print(output)
        #print("")
        messages.append({"role": "assistant", "content": output})
        curAction = output[output.find("Action: ") + len("Action: "):]
        curAction = curAction.lower()
        expression = curAction[curAction.find('[') + 1:curAction.find(']')]
        expression = expression.replace(',', '') # Commas are interpreted as tuples, so we remove them to avoid errors with eval
        #print(curAction)
        if "finish" in curAction:
            finished = True
            try:
                curResponse["received"] = int(eval(expression))
            except:
                print("#################################################")
                print("Finish Error")
                print("Question " + str(counter))
                print(output)
                curResponse["received"] = -987654321
        else:
            expression = expression.split(';')
            evaled = None
            try:
                for i in range(len(expression)):
                    expression[i] = float(expression[i])
            except:
                messages.append({"role": "user", "content": "Observation: That was not a valid action. Please try again. REMINDER: You cannot put variables as inputs to actions."})
                continue
            if "add" in curAction:
                evaled = str(expression[0] + expression[1])
            elif "sub" in curAction:
                evaled = str(expression[0] - expression[1])
            elif "mult" in curAction:
                evaled = str(expression[0] * expression[1])
            elif "div" in curAction:
                evaled = str(expression[0] / expression[1])
            elif "sqrt" in curAction:
                evaled = str(math.sqrt(expression[0]))
            else:
                evaled = "That was not a valid calculation. Please input a valid one."
            messages.append({"role": "user", "content": "Observation: " + evaled})
            #print(messages[-1]["content"])
    curResponse["messages"] = messages
    responses.append(curResponse)
    counter += 1

numCorrect = 0
for i in range(len(responses)):
    response = responses[i]
    #print(i)
    #print("Target: " + str(response["target"]))
    #print("Received: " + str(response["received"]))
    #print("")
    if response["target"] == response["received"]:
        numCorrect += 1
print("Number correct = " + str(numCorrect))

[173, 392, 1152, 746, 867, 225, 549, 698, 412, 252, 136, 456, 193, 873, 411, 732, 1011, 874, 1008, 536, 251, 824, 119, 1246, 1201, 929, 608, 265, 469, 524, 1233, 1060, 484, 478, 1199, 691, 454, 288, 382, 1229, 237, 555, 276, 993, 556, 59, 35, 701, 892, 563, 570, 1093, 320, 859, 1132, 586, 712, 1196, 489, 444, 1222, 50, 948, 61, 668, 1283, 904, 999, 1294, 324, 118, 492, 1041, 722, 1055, 569, 988, 834, 651, 295, 1113, 566, 30, 236, 21, 396, 211, 437, 447, 307, 675, 77, 1215, 1000, 960, 499, 284, 175, 653, 1252]


Going through math questions:   0%|          | 0/1319 [00:00<?, ?it/s]

#################################################
Finish Error
Question 906
Thought: After using 3 red shoeboxes, Tim has 6 red shoeboxes left. Now we have the final counts: Tim has 4 blue shoeboxes and 6 red shoeboxes remaining. 
Action: FINISH[4 blue and 6 red]
#################################################
Finish Error
Question 1001
Thought: So, the total time required to make, bake, cool, and frost the cake is 180 minutes. Now, we need to determine the latest time Jordan can start making the cake to have it ready by 5:00 PM. 

180 minutes is equivalent to 3 hours (since 180 divided by 60 equals 3). Therefore, we need to subtract 3 hours from 5:00 PM to find the latest start time. 

5:00 PM minus 3 hours is 2:00 PM. 
Action: FINISH[2:00 PM]
#################################################
Finish Error
Question 1017
Thought: Thus, Olivia has 18 portraits. To summarize, she has 27 selfies and 18 portraits. 
Action: FINISH[27 selfies and 18 portraits]
Number correct = 1214


In [30]:
for message in responses[18]["messages"][len(gsm_messages):]:
    print(message["content"])
'''
for i in range(len(responses)):
    #if responses[i]["target"] != responses[i]["received"]:
    response = responses[i]
    print(i)
    print("Target: " + str(responses[i]["target"]))
    print("Received: " + str(responses[i]["received"]))
    print("")
    for message in response["messages"][len(aqua_messages):]:
        print(message["content"])
    print("\n")
'''

Whenever you synthesize information and perform reasoning, it is a Thought. When you need to calculate a value, e.g., 5+3, it is an Action. There are 6 types of actions: 
(1)ADD[a;b]: ADD takes in two numbers a and b, and returns their sum. For example, ADD[7;3] will return the value of 7+3.
(2)SUB[a;b]: SUB takes in two numbers a and b, and returns their difference. For example, SUB[6;2] will return the value of 6-2.
(3)MULT[a;b]: MULT takes in two numbers a and b, and returns their product. For example, MULT[8;5] will return the value of 8*5.
(4)DIV[a;b]: DIV takes in two numbers a and b, and returns their quotient. For example, DIV[9;3] will return 9/3.
(5)SQRT[a]: SQRT takes in a number a, and returns its square root. For example, SQRT[25] will return the square root of 25, i.e., 5.
(6)FINISH[answer]FINISH: When you finish a question, write your answer as FINISH[answer] and terminate your response.
Terminate your response after an action. Then, an Observation is returned by the use

'\nfor i in range(len(responses)):\n    #if responses[i]["target"] != responses[i]["received"]:\n    response = responses[i]\n    print(i)\n    print("Target: " + str(responses[i]["target"]))\n    print("Received: " + str(responses[i]["received"]))\n    print("")\n    for message in response["messages"][len(aqua_messages):]:\n        print(message["content"])\n    print("\n")\n'